In [158]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


# Initialize Data Frame

In [159]:
testRaw = pd.read_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/test.csv')
testRaw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


##  Find NAs

In [160]:
testRaw.isna().sum(axis=0)

Id                  0
MSSubClass          0
MSZoning            4
LotFrontage       227
LotArea             0
Street              0
Alley            1352
LotShape            0
LandContour         0
Utilities           2
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         1
Exterior2nd         1
MasVnrType         16
MasVnrArea         15
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           44
BsmtCond           45
BsmtExposure       44
BsmtFinType1       42
BsmtFinSF1          1
BsmtFinType2       42
BsmtFinSF2          1
BsmtUnfSF           1
TotalBsmtSF         1
Heating             0
HeatingQC           0
CentralAir          0
Electrical          0
1stFlrSF            0
2ndFlrSF            0
LowQualFin

## Action Plan for NA values

**Cleaning NAs is going to be a process**  
+ All variables with quality in the description use NA to denote there is no object to denote the quality of  
    - suggest replacing with text 'none' and using that as the default case in dummification   
    - build a masking variable
+ MSZoning has 4 NA values, no easily identifiable characteristics to interpolate, strongly consider dropping. Could be a very important variable.
+ LotFrontage Variable has 227 NA values, but no values equal to zero, find way to impute, see below for special treatment
+ Utilities has 2 NA values, no identifying charactersitics for this variable strongly consider dropping  
+ Exterior1st has 1 NA value, convert to 'Other' 
+ Exterior2nd has 1 NA value, convert to 'Other' 
+ MasVnrArea has 16 NA values, convert to None  
+ MasVnrQual has 15 NA values corresponding to the NAs in previous category, change to 0 (an acceptable value in this dataset)
+ BsmtFinSF1 has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtFinSF2 has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtUnfSF has 1 NA value, change to 0 (see anaomlies section for explanation)  
+ BsmtFullBath has 2 NAs, but records say there is no basement, change to zero  
+ BsmtHalfBath has 2 NAs, but records say there is no basement, change to zero 
+ KitchenAbvGr has 1 NA, but has a kitchen listed in the record, no way to interpolate, consider dropping
+ Functional has 2 NA values, no way to interpolate consider dropping
+ GarageCars, and GarageArea NA values will be dropped (see below anomoly description)
+ SaleType, has 1 NA value, change to 'Oth'

**Variables with NA values who's rows we should drop**  
+ MSZoning  
+ Utilities
+ KitchenQual
+ Functional




### Mask Variable for replacing NA values in columns where NA has meaning

In [161]:
naMask = ['Alley', 'Exterior1st','Exterior2nd', 'BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish'\
         ,'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature' ]

### Additional Data Anomolies

**Mismatch in Basement Existence Data**

In [162]:
testRaw[testRaw['BsmtQual'].isna()].loc[:,['BsmtQual', 'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1','BsmtFinType2', 'BsmtFinSF2','BsmtUnfSF', 'TotalBsmtSF' ]]\
.loc[testRaw['BsmtUnfSF']!=0]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF
660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
757,NaN,Fa,No,Unf,0.0,Unf,0.0,173.0,173.0
758,NaN,TA,No,Unf,0.0,Unf,0.0,356.0,356.0


These 3 records have basement quality listed as 'No Basment', the last two records however have values listed for basement features, can not interpolate basement quality from data recomend dropping records 757 and 758.
For record 660 Set basement square footage columns to 0, replacing NA values to NONE in previous step will take care of the rest.

**Missing Column, Kitchen**

In the data dictionary file, a column named 'Kitchen' is listed, however the datafile has a column named 'KitchenAbvGr' instead

**Garage Existence Data Mismatch**

In [163]:
testRaw[testRaw['GarageYrBlt'].isna()].loc[~testRaw['GarageType'].isna(), ['GarageType','GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'GarageYrBlt']]

,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,GarageYrBlt
666,Detchd,NaN,1.0,360.0,NaN,NaN,NaN
1116,Detchd,NaN,NaN,NaN,NaN,NaN,NaN


2 records with garage existence listed but limited values listed for the rest with no legal option for NaN, consider dropping records 666, 1116

**MSZoning Values, looking for relationships to other columns**

In [164]:
testRaw.loc[testRaw['MSZoning'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
455,1916,30,NaN,109.0,21780,Grvl,NaN,Reg,Lvl,NaN,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,2,4,1910,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Fa,Fa,CBlock,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,TA,N,FuseA,810,0,0,810,0.0,0.0,1,0,1,1,TA,4,Min1,0,NaN,Detchd,1975.0,Unf,1.0,280.0,TA,TA,N,119,24,0,0,0,0,NaN,NaN,NaN,0,3,2009,ConLD,Normal
756,2217,20,NaN,80.0,14584,Pave,NaN,Reg,Low,AllPub,Inside,Mod,IDOTRR,Norm,Norm,1Fam,1Story,1,5,1952,1952,Gable,CompShg,AsbShng,VinylSd,None,0.0,Fa,Po,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,Wall,Po,N,FuseA,733,0,0,733,0.0,0.0,1,0,2,1,Fa,4,NaN,0,NaN,Attchd,1952.0,Unf,2.0,487.0,Fa,Po,N,0,0,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Abnorml
790,2251,70,NaN,NaN,56600,Pave,NaN,IR1,Low,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,2.5Unf,5,1,1900,1950,Hip,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,686.0,686.0,GasA,Ex,Y,SBrkr,1150,686,0,1836,0.0,0.0,2,0,4,1,TA,7,Maj1,0,NaN,Detchd,1900.0,Unf,1.0,288.0,TA,Fa,N,0,0,0,0,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal
1444,2905,20,NaN,125.0,31250,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Artery,Norm,1Fam,1Story,1,3,1951,1951,Gable,CompShg,CBlock,VinylSd,None,0.0,TA,Fa,CBlock,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,TA,Y,FuseA,1600,0,0,1600,0.0,0.0,1,1,3,1,TA,6,Mod,0,NaN,Attchd,1951.0,Unf,1.0,270.0,Fa,TA,N,0,0,135,0,0,0,NaN,NaN,NaN,0,5,2006,WD,Normal


In [165]:
testRaw.loc[testRaw['Neighborhood'] == 'IDOTRR', ['MSZoning', 'Neighborhood']].value_counts()

MSZoning  Neighborhood
RM        IDOTRR          40
C (all)   IDOTRR          13
dtype: int64

In [166]:
testRaw.loc[testRaw['Neighborhood'] == 'Mitchel', ['MSZoning', 'Neighborhood']].value_counts()

MSZoning  Neighborhood
RL        Mitchel         60
RM        Mitchel          4
dtype: int64

In [167]:
testRaw.MSZoning.value_counts()

RL         1114
RM          242
FV           74
C (all)      15
RH           10
Name: MSZoning, dtype: int64

In [168]:
testRaw.loc[testRaw['MSZoning'] == 'RM'].LotArea.describe()

count      242.000000
mean      6418.673554
std       3381.854064
min       1470.000000
25%       4250.500000
50%       6120.000000
75%       8603.000000
max      33120.000000
Name: LotArea, dtype: float64

In [169]:
testRaw.loc[testRaw['MSZoning'] == 'C (all)'].LotArea.describe()

count       15.000000
mean      7533.666667
std       2416.430504
min       3300.000000
25%       5670.000000
50%       8250.000000
75%       9000.000000
max      12366.000000
Name: LotArea, dtype: float64

**Can't find consistent secondary variables, will fill with most common Zone type in that neighborhood**

In [170]:
testRaw.loc[(testRaw['MSZoning'].isna()) & (testRaw['Neighborhood']== 'IDOTRR'), ['MSZoning']] = 'RM'
testRaw.loc[(testRaw['MSZoning'].isna()) & (testRaw['Neighborhood']== 'Mitchel'), ['MSZoning']] = 'RL'
testClean= testRaw

## Removing NAs in Utilities Columns

In [171]:
testClean.loc[testClean['Utilities'].isna(), ['Utilities']] = 'AllPub'

### Removing NAs in Kitchen Quality columns

In [172]:
testClean.loc[testClean['KitchenQual'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
95,1556,50,RL,72.0,10632,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,ClearCr,Norm,Norm,1Fam,1.5Fin,5,3,1917,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,Gd,Fa,No,Unf,0.0,Unf,0.0,689.0,689.0,GasA,Gd,N,SBrkr,725,499,0,1224,0.0,0.0,1,1,3,1,NaN,6,Mod,0,NaN,Detchd,1917.0,Unf,1.0,180.0,Fa,Fa,N,0,0,248,0,0,0,NaN,NaN,NaN,0,1,2010,COD,Normal


In [173]:
testClean.KitchenQual.value_counts()

TA    757
Gd    565
Ex    105
Fa     31
Name: KitchenQual, dtype: int64

In [174]:
testClean.loc[testClean['KitchenQual'].isna(), ['KitchenQual']] = 'TA'

In [175]:
### Removing NAs in Functional columns

In [176]:
testClean.loc[testClean['Functional'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
756,2217,20,RM,80.0,14584,Pave,NaN,Reg,Low,AllPub,Inside,Mod,IDOTRR,Norm,Norm,1Fam,1Story,1,5,1952,1952,Gable,CompShg,AsbShng,VinylSd,None,0.0,Fa,Po,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,Wall,Po,N,FuseA,733,0,0,733,0.0,0.0,1,0,2,1,Fa,4,NaN,0,NaN,Attchd,1952.0,Unf,2.0,487.0,Fa,Po,N,0,0,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Abnorml
1013,2474,50,RM,60.0,10320,Pave,Grvl,Reg,Lvl,AllPub,Corner,Gtl,IDOTRR,Artery,Norm,1Fam,1.5Fin,4,1,1910,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Fa,Fa,CBlock,TA,Fa,No,Unf,0.0,Unf,0.0,771.0,771.0,GasA,Fa,Y,SBrkr,866,504,114,1484,0.0,0.0,2,0,3,1,TA,6,NaN,0,NaN,Detchd,1910.0,Unf,1.0,264.0,TA,Fa,N,14,211,0,0,84,0,NaN,NaN,NaN,0,9,2007,COD,Abnorml


In [177]:
testClean.Functional.value_counts()

Typ     1357
Min2      36
Min1      34
Mod       20
Maj1       5
Maj2       4
Sev        1
Name: Functional, dtype: int64

In [178]:
testClean.loc[testClean['Functional'].isna(), ['Functional']] = 'Typ'

## Removing NAs from Garage Cars and Garage Area

In [179]:
testClean.loc[testClean['GarageCars'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1116,2577,70,RM,50.0,9060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,2Story,5,6,1923,1999,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,BrkTil,Gd,TA,No,ALQ,548.0,Unf,0.0,311.0,859.0,GasA,Ex,Y,SBrkr,942,886,0,1828,0.0,0.0,2,0,3,1,Gd,6,Typ,0,NaN,Detchd,NaN,NaN,NaN,NaN,NaN,NaN,Y,174,0,212,0,0,0,NaN,MnPrv,NaN,0,3,2007,WD,Alloca


**All garage values are none besides for GarageType. There must not be a garage**
**Change NAs to 0 and type to None**

In [180]:
testClean.loc[testClean['GarageCars'].isna(), ['GarageCars', 'GarageArea', 'GarageType']] = [0,0,'None']

# Clean Data

## Change NA value of certain columns where NA has meaning to None

In [181]:
for i in range(len(naMask)):
    testClean[naMask[i]].fillna('None', inplace = True)

## Change Specific NA values depending on Context  

**Exterior1st has 1 NA value, convert to 'Other'**

In [182]:
testClean.Exterior1st.fillna('Other', inplace = True)

**Exterior2nd has 1 NA value, convert to 'Other'**

In [183]:
testClean.Exterior2nd.fillna('Other', inplace = True)

**MasVnrType has 16 NA values, convert to None**

In [184]:
testClean.MasVnrType.fillna('None', inplace = True)

**MasVnrArea has 15 NA values corresponding to the NAs in previous category, change to 0**

In [185]:
testClean.MasVnrArea.fillna(0, inplace = True)

**TotalBsmtSF has 1 NA value, change to 0**

In [186]:
testClean.TotalBsmtSF.fillna(0, inplace = True)

**BsmtFinSF1 has 1 NA value, change to 0**

In [187]:
testClean.BsmtFinSF1.fillna(0, inplace = True)

**BsmtFinSF2 has 1 NA value, change to 0**

In [188]:
testClean.BsmtFinSF2.fillna(0, inplace = True)

**BsmtUnfSF has 1 NA value, change to 0**

In [189]:
testClean.BsmtUnfSF.fillna(0, inplace = True)

**BsmtFullBath has 2 NAs, but records say there is no basement, change to zero**

In [190]:
testClean.BsmtFullBath.fillna(0, inplace = True)

**BsmtHalfBath has 2 NAs, but records say there is no basement, change to zero**

In [191]:
testClean.BsmtHalfBath.fillna(0, inplace = True)

**SaleType, has 1 NA value, change to 'Oth'**

In [192]:
testClean.SaleType.fillna('Oth', inplace = True)

#### LotFrontage computations

In [193]:
testClean.corr().LotFrontage.sort_values()

MSSubClass      -0.450505
OverallCond     -0.092312
BsmtHalfBath    -0.043422
3SsnPorch       -0.037487
LowQualFinSF    -0.037294
2ndFlrSF        -0.036185
YrSold          -0.025263
MoSold           0.008810
Id               0.009930
EnclosedPorch    0.013340
KitchenAbvGr     0.016072
HalfBath         0.023850
BsmtFinSF2       0.046460
MiscVal          0.068161
BsmtUnfSF        0.090476
YearRemodAdd     0.092603
ScreenPorch      0.113444
YearBuilt        0.122356
BsmtFullBath     0.124377
PoolArea         0.134232
WoodDeckSF       0.157426
FullBath         0.163078
OpenPorchSF      0.179795
OverallQual      0.182870
BsmtFinSF1       0.203383
BedroomAbvGr     0.205100
MasVnrArea       0.248433
Fireplaces       0.257037
TotalBsmtSF      0.312350
GarageCars       0.337101
TotRmsAbvGrd     0.344366
GrLivArea        0.357125
GarageArea       0.376219
1stFlrSF         0.461239
LotArea          0.644608
LotFrontage      1.000000
Name: LotFrontage, dtype: float64

**LotArea has the highest positive correlation, and MSSubClass has the highest negative correlation**
**Perhaps if we group by Subclass and regress with LotArea we can get good values for LotFrontage**

In [194]:
A = testClean.loc[testClean['LotFrontage'].notna(),['MSSubClass', 'LotFrontage', 'LotArea']]

In [195]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm.fit(A[['LotArea']], A[['LotFrontage']])
RSquared = lm.score(A[['LotArea']], A[['LotFrontage']])
Coefficient = lm.coef_
print(f'This model has a score of {RSquared} and a coefficient of {Coefficient}')

This model has a score of 0.41552011540471123 and a coefficient of [[0.00320437]]


**That is bad, let's try grouping by MSSubClass**

In [196]:
AGroup = A.groupby('MSSubClass')
for name,group in AGroup:
    lm = linear_model.LinearRegression()
    lm.fit(group[['LotArea']], group[['LotFrontage']])
    RSquared = lm.score(group[['LotArea']], group[['LotFrontage']])
    Coefficient = lm.coef_
    print(f'This model for MSSubClass = {name} has a score of {RSquared} and a coefficient of {Coefficient}\n\
    There are {group.shape[0]} records in this group')


This model for MSSubClass = 20 has a score of 0.20106037718514003 and a coefficient of [[0.00178453]]
    There are 457 records in this group
This model for MSSubClass = 30 has a score of 0.48276089512483367 and a coefficient of [[0.00265743]]
    There are 63 records in this group
This model for MSSubClass = 40 has a score of 1.0 and a coefficient of [[0.00219573]]
    There are 2 records in this group
This model for MSSubClass = 45 has a score of 0.7935575128584776 and a coefficient of [[0.00544202]]
    There are 6 records in this group
This model for MSSubClass = 50 has a score of 0.3348910902771972 and a coefficient of [[0.00215053]]
    There are 134 records in this group
This model for MSSubClass = 60 has a score of 0.12886444561771293 and a coefficient of [[0.00257031]]
    There are 212 records in this group
This model for MSSubClass = 70 has a score of 0.5048441086785984 and a coefficient of [[0.00297606]]
    There are 62 records in this group
This model for MSSubClass = 75 

**Quick Idea, try grouping by subclass and lotshape

In [197]:
B = testClean.loc[testClean['LotFrontage'].notna(),['MSSubClass', 'LotShape', 'LotFrontage', 'LotArea']]

In [198]:
BGroup = B.groupby(['LotShape', 'MSSubClass'])
for name,group in BGroup: 
    lm = linear_model.LinearRegression()
    lm.fit(group[['LotArea']], group[['LotFrontage']])
    RSquared = lm.score(group[['LotArea']], group[['LotFrontage']])
    Coefficient = lm.coef_
    print(f'This model for LotShape, MSSubClass {name} has a score of {RSquared} and a coefficient of {Coefficient}\n\
    There are {group.shape[0]} records in this group')

This model for LotShape, MSSubClass ('IR1', 20) has a score of 0.01997332039363031 and a coefficient of [[0.0004845]]
    There are 133 records in this group
This model for LotShape, MSSubClass ('IR1', 30) has a score of 1.0 and a coefficient of [[0.0026441]]
    There are 2 records in this group
This model for LotShape, MSSubClass ('IR1', 50) has a score of 0.03516547787909896 and a coefficient of [[0.00101028]]
    There are 8 records in this group
This model for LotShape, MSSubClass ('IR1', 60) has a score of 0.12634836389127224 and a coefficient of [[0.00275212]]
    There are 104 records in this group
This model for LotShape, MSSubClass ('IR1', 70) has a score of 0.11231193776683412 and a coefficient of [[0.00168477]]
    There are 5 records in this group
This model for LotShape, MSSubClass ('IR1', 80) has a score of 0.10185133528055745 and a coefficient of [[0.00188946]]
    There are 21 records in this group
This model for LotShape, MSSubClass ('IR1', 85) has a score of 0.229296

C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\jmeis\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:796: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn


This model for LotShape, MSSubClass ('Reg', 80) has a score of 0.2570455005519471 and a coefficient of [[0.00407744]]
    There are 25 records in this group
This model for LotShape, MSSubClass ('Reg', 85) has a score of 0.2744647810185942 and a coefficient of [[0.00499288]]
    There are 15 records in this group
This model for LotShape, MSSubClass ('Reg', 90) has a score of 0.11847959846543432 and a coefficient of [[0.00243286]]
    There are 40 records in this group
This model for LotShape, MSSubClass ('Reg', 120) has a score of 0.6365285927030471 and a coefficient of [[0.00452798]]
    There are 34 records in this group
This model for LotShape, MSSubClass ('Reg', 160) has a score of 0.6472480484930114 and a coefficient of [[0.00523689]]
    There are 59 records in this group
This model for LotShape, MSSubClass ('Reg', 180) has a score of 0.9774721109065309 and a coefficient of [[0.00685945]]
    There are 6 records in this group
This model for LotShape, MSSubClass ('Reg', 190) has a

**Hrmm...rSquared for regualr lot sizes seem to be higher, let's single those out**

In [199]:
C = testClean.loc[(testClean['LotFrontage'].notna()) & (testClean['LotShape'] == 'Reg'),['MSSubClass', 'LotShape', 'LotFrontage', 'LotArea']]
C

,MSSubClass,LotShape,LotFrontage,LotArea
0,20,Reg,80.0,11622
8,20,Reg,85.0,10176
9,20,Reg,70.0,8400
11,160,Reg,21.0,1680
12,160,Reg,21.0,1680
...,...,...,...,...
1454,160,Reg,21.0,1936
1455,160,Reg,21.0,1894
1456,20,Reg,160.0,20000
1457,85,Reg,62.0,10441


In [200]:
CGroup = C.groupby('MSSubClass')
CfitList = []
for name,group in CGroup: 
    lm = linear_model.LinearRegression()
    lm.fit(group[['LotArea']], group[['LotFrontage']])
    RSquared = round(lm.score(group[['LotArea']], group[['LotFrontage']]),5)
    Coefficient = round(lm.coef_[0][0],5)
    print(f'This model for MSSubClass = {name} has a score of {RSquared} and a coefficient of {Coefficient}\n\
    There are {group.shape[0]} records in this group')
    CfitList.append([name, RSquared, Coefficient])
rSum = 0
for i in CfitList:
    rSum = rSum + i[1]
print(f'The average R Squared for this model is {rSum/len(CfitList)}')

This model for MSSubClass = 20 has a score of 0.58433 and a coefficient of 0.00365
    There are 317 records in this group
This model for MSSubClass = 30 has a score of 0.51145 and a coefficient of 0.00274
    There are 60 records in this group
This model for MSSubClass = 40 has a score of 1.0 and a coefficient of 0.0022
    There are 2 records in this group
This model for MSSubClass = 45 has a score of 0.79356 and a coefficient of 0.00544
    There are 6 records in this group
This model for MSSubClass = 50 has a score of 0.38944 and a coefficient of 0.00258
    There are 125 records in this group
This model for MSSubClass = 60 has a score of 0.51905 and a coefficient of 0.00501
    There are 99 records in this group
This model for MSSubClass = 70 has a score of 0.5321 and a coefficient of 0.00301
    There are 57 records in this group
This model for MSSubClass = 75 has a score of 0.01211 and a coefficient of 0.00022
    There are 6 records in this group
This model for MSSubClass = 80 

**Whats the breakdown of lotShape and MSSubClass for the NA values in LotFrontage**

In [201]:
D = testClean.loc[testClean['LotFrontage'].isna(),['MSSubClass', 'LotShape']]
Dmelt = pd.melt(D)
Dmelt.groupby(['variable', 'value']).size()

variable    value
LotShape    IR1      154
            IR2       13
            IR3        2
            Reg       58
MSSubClass  20        86
            30         7
            50         9
            60        64
            70         6
            75         1
            80        13
            85        10
            90        12
            120       12
            150        1
            160        4
            180        1
            190        1
dtype: int64

**These groupbys are going nowhere and I'm not find any consistent strong correlations based on other variables besides LotArea**

**#@!$ it, I'm just multiplying the average ratio of LotArea and LotFrontage to impute the value of LotArea**

In [202]:
E = testClean.loc[testClean['LotFrontage'].notna(),['LotArea', 'LotFrontage']]
fRatio = E.agg('mean')[1]/ E.mean()[0]

In [203]:
testClean.loc[testClean['LotFrontage'].isna(),'LotFrontage'] = testClean['LotArea'] * fRatio

## Check to see if we elimanted all NA values

In [204]:
testClean.isna().sum(axis=0)

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
MasVnrArea       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinSF1       0
BsmtFinType2     0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
Heating          0
HeatingQC        0
CentralAir       0
Electrical       0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr

## A Thing of Beauty Emerges, no NA values, save to new csv

In [205]:
testClean.to_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/testZeroNAs.csv')

# Repeat Process for Train Dataset

## Initialize Dataset

In [206]:
trainRaw = pd.read_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/train.csv')
trainRaw.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Eliminate NA values

**We know certain columns' NA has meaning, so we are going to change those NA values to 'None'**

In [207]:
trainClean = trainRaw
for i in range(len(naMask)):
    trainClean[naMask[i]].fillna('None', inplace = True)
trainClean.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000


**Find Remaining NA Values**

In [208]:
A = trainClean.isna().sum(axis=0)
A[A>0]

LotFrontage    259
MasVnrType       8
MasVnrArea       8
Electrical       1
dtype: int64

Using Previous Logic:  
+ LotFrontage NAs will be converted to the avaerage ration of LotFrontage and  LotArea
+ Convert MasVnrType and MasVnrArea Nas to None and 0 respectively  

The one NA with an unknown eletrical system will be need to be investigated.

In [209]:
trainClean.MasVnrType.fillna('None', inplace = True)
trainClean.MasVnrArea.fillna(0, inplace = True)


In [210]:
trainClean.loc[trainClean['LotFrontage'].isna(),'LotFrontage'] = trainClean['LotArea'] * fRatio

**lets look at the distribution of the values in the electrical column**

In [211]:
trainClean.Electrical.value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

**Very lopsided distribution, I feel asigning the NA to the most common value is the right choice**  
The slight possibility of  introduced error from the value change IMO outweighs the loss of a record in the train data

In [212]:
trainClean.loc[trainClean['Electrical'].isna(), 'Electrical'] = 'SBrkr'    

In [213]:
A = trainClean.isna().sum(axis=0)
A[A>0]

Series([], dtype: int64)

**No More NAs**.....whew!
## Write to new CSV

In [214]:
trainClean.to_csv('c:/Users/jmeis/NYC_DSA/HousingPrices/Data/trainZeroNAs.csv')